In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import mwxml
from src.mongo_utils import collection, extract_content
from tqdm.notebook import tqdm

In [3]:
def extract_text(page):
    """Extracts the text content from a MediaWiki page."""
    for revision in page:
        if hasattr(revision, 'text') and revision.text is not None:
            return revision.text
        elif hasattr(revision, 'slots') and 'main' in revision.slots:
            return revision.slots['main'].text

def process_dump(xml_dump_path):
    dump = mwxml.Dump.from_file(open(xml_dump_path, 'rb'))

    for page in tqdm(dump):
        text = extract_text(page)
        if text:
            article_data = {
                'title': page.title,
                'text': text
            }
            collection.insert_one(article_data)

xml_dump_path = "dewiktionary-20240101-pages-articles.xml"

process_dump(xml_dump_path)

In [27]:
import re
from collections import defaultdict

def get_entries(text, token):
    if token not in text:
        return []
    search_token = '{{' + token + '}}'
    
    m = re.search(search_token+'\s*:(.*?)(?=\n{{[^{}]+}}|\Z)', text, re.DOTALL)
    if not m:
        return []
    m = m.group()
    m = m.replace(search_token+'\n', '')
    lines = list(filter(lambda item: item.strip(), m.split("\n")))
    return lines

def extract_entries(entries):
    curr_id = -1
    bedeutungen = defaultdict(lambda: [])
    for i, line in enumerate(entries):
        if re.search('{{Beispiele fehlen.*}}', line):
            continue
        if m := re.search(':\[[0-9]+\]', line):
            curr_id = int(re.search('[0-9]+', m.group()).group())
            entry = line.replace(f':[{curr_id}]', '').strip()
            if entry:
                bedeutungen[str(curr_id)] += [entry]
        elif m := re.search('::— ', line):
            if curr_id != -1:
                bedeutungen[str(curr_id)] += [line.replace('::— ', '').strip()]
    return bedeutungen

In [ ]:
from tqdm.notebook import tqdm


search_query =  {'text': {"$regex": ".*{{Beispiele}}.*", "$options": "i"}}
result = collection.find(search_query)
count = 0

empty_words = []
available_words = []

i = 0
for doc in tqdm(result):
    insert_data = {}
    a = get_entries(doc['text'], 'Bedeutungen')
    bedeutungen = extract_entries(a)
    insert_data['bedeutungen'] = bedeutungen
    a = get_entries(doc['text'], 'Beispiele')
    beispiele = extract_entries(a)
    insert_data['beispiele'] = beispiele

    if beispiele and bedeutungen:
        available_words += [doc['title']]
    else:
        empty_words += [doc['title']]
        continue
    result = collection.update_one({"_id": doc['_id']}, {'$set': insert_data})
count

In [5]:
from tqdm.notebook import tqdm
from src.mongo_utils import clear_database

clear_database()

search_query =  {'text': {"$regex": ".*{{Beispiele}}.*", "$options": "i"}}
result = collection.find(search_query)
count = 0

empty_words = []
available_words = []

i = 0
for doc in tqdm(result):
    insert_data = {}
    bedeutungen, beispiele = extract_content(doc['text'])
    insert_data['bedeutungen'] = bedeutungen
    insert_data['beispiele'] = beispiele

    if beispiele and bedeutungen:
        count += 1
        available_words += [doc['title']]
    else:
        empty_words += [doc['title']]
        continue
    result = collection.update_one({"_id": doc['_id']}, {'$set': insert_data})
count

In [113]:
extract_content(doc['text'])

In [26]:
beispiele

In [8]:
bedeutungen

In [260]:
doc

In [263]:
len(available_words)

In [264]:
len(empty_words)

In [126]:
example_1 = str(doc['text'])

In [5]:


a = get_entries(doc['text'], 'Bedeutungen')
bedeutungen = extract_entries(a)
a = get_entries(doc['text'], 'Beispiele')
beispiele = extract_entries(a)

print(bedeutungen)
print(beispiele)

In [258]:
a

In [225]:
import mwparserfromhell

wikicode = mwparserfromhell.parse(doc['text'])
for template in wikicode.filter_templates():
    if template.name.matches("Bedeutungen"):
        bedeutungen_template = template
        break
    else:
        print(template.name)

In [224]:
print(doc['text'])

In [100]:
for line in a:
    print(line)

In [83]:
repr(re.search('[0-9]+', ':[11]').group())